In [1]:
import keras
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import count
from sklearn.metrics import accuracy_score
from keras.datasets import cifar10
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from scipy.stats import pearsonr
from tqdm import tqdm
from scipy import ndimage
from IPython.display import clear_output

Using TensorFlow backend.


In [0]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 28
NUM_CLASSES = 10
NUM_CHANNELS = 1
MODEL_NAME = "MNIST_data_augmentation"
PATH = ""
NR_OF_RUNS = 10

# Set seeds

# Preprocess

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = preprocess(x_train)
x_test = preprocess(x_test)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_trainc = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
x_train_full = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train_full /= 255
x_test /= 255

# Model

In [0]:
def MNISTmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', strides = 2)(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size = (2,2), strides=(2,2), padding = "same")(x)
    x = Conv2D(filters=32, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters = 10, kernel_size = (1,1),strides = (1,1), padding = 'valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

#Predict


In [0]:
def hard_voting(models, X):
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(X), axis=1))

    prediction = np.transpose(predictions)
    prediction = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=prediction)

    return prediction

def soft_voting(models, X):
    predictions = np.empty((len(X),0,NUM_CLASSES))

    for m in models:
        pred = np.expand_dims(m.predict(X), axis=1)
        predictions = np.append(predictions, pred, axis=1)

    predictions = np.apply_along_axis(np.transpose, axis=1, arr=predictions)
    predictions = np.mean(predictions, axis=1)
    prediction = np.argmax(predictions, axis=1)

    return prediction

def predict(models, X, Y,voting = 'hard'):
    
    if voting == "soft":
      prediction = soft_voting(models, X)
    elif voting == "hard":
      prediction = hard_voting(models, X)
    else:
      raise ValueError(f"Voting mechanism: {VOTING} not supported")

    return accuracy_score(prediction, np.argmax(Y, axis=1))

#Augmentation 


In [0]:
def rotate_image(image):
    if np.random.rand() < 0.2:
        angles = np.linspace(1,10,10)
        rotation_angle = np.random.choice(angles)
        if np.random.rand() < 0.5:
            image = ndimage.rotate(image, rotation_angle, reshape = False)
        else:
            image = ndimage.rotate(image, -rotation_angle, reshape = False) 
    return image

In [0]:
def horizontal_shift(image):
    if np.random.rand() < 0.2:
        num_rows, num_cols = image.shape[:2]
        if np.random.rand() < 0.5:
            translation_matrix = np.float32([ [1,0,3], [0,1,0] ])
            image = cv2.warpAffine(image, translation_matrix, (num_cols, num_rows))
            #image = image.reshape((IMAGE_SIZE,IMAGE_SIZE,1))
        else:
            translation_matrix = np.float32([ [1,0,-3], [0,1,0] ])
            image = cv2.warpAffine(image, translation_matrix, (num_cols, num_rows))
            #image = image.reshape((IMAGE_SIZE,IMAGE_SIZE,1))
    return image

In [0]:
def vertical_shift(image):
    if np.random.rand() < 0.2:
        num_rows, num_cols = image.shape[:2]
        if np.random.rand() < 0.5:
            translation_matrix = np.float32([ [1,0,0], [0,1,2] ])
            image = cv2.warpAffine(image, translation_matrix, (num_cols, num_rows))
            #image = image.reshape((IMAGE_SIZE,IMAGE_SIZE,1))
        else:
            translation_matrix = np.float32([ [1,0,0], [0,1,-2] ])
            image = cv2.warpAffine(image, translation_matrix, (num_cols, num_rows))
            #image = image.reshape((IMAGE_SIZE,IMAGE_SIZE,1))
    return image

In [0]:
funcs = [rotate_image,
         horizontal_shift, 
         vertical_shift]

In [0]:
augmentations = ["rotation", "horizontal shifting", "vetical shifting"]

# Train

In [14]:
for run in range(1, NR_OF_RUNS+1):
    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_trainc, test_size=0.20, shuffle= True)
    
    models = []
    accuracies = []
    predictions = []
    for i in range(len(funcs)):
        print(f"\n ===== Train model: Data augmentation method: {augmentations[i]}  =====")
        
        # Set the seeds
        np.random.seed(run*i)
        tf.random.set_seed(run*i)

        # Create directories
        os.makedirs(PATH + MODEL_NAME + f"/{run}/history", exist_ok=True)
        os.makedirs(PATH + MODEL_NAME + f"/{run}/weights", exist_ok=True)
        
        #data augmentation function
        preprocessing_function = funcs[i]
        #data augmentation generator
        datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)
        datagen = datagen.flow(x_train,y_train, batch_size = BATCH_SIZE)
        # weight init method
        model = CIFARmodel(IMAGE_SIZE,NUM_CLASSES,NUM_CHANNELS)
    
        #save weights 
        weights_path = PATH + MODEL_NAME + f"/{run}/weights/weights-{augmentations[i]}.h5"
            
        #save weights 
        weights_path = PATH + MODEL_NAME + f"/{run}/weights/weights-{augmentations[i]}.h5"
        if os.path.exists(weights_path):
            print(f"Skipping training of model {augmentations[i]}: weights exists")
            model.load_weights(weights_path)
        else:
            # initiate early stopping
            es = EarlyStopping(min_delta=0.01, patience=3)
            csv_logger = CSVLogger(PATH + MODEL_NAME + f"/{run}/history/history-{augmentations[i]}.csv", separator=';')
            #train
            model.fit_generator(datagen,
                                epochs = EPOCHS,
                                validation_data = (x_val,y_val),
                                shuffle = True,
                                callbacks=[es, csv_logger])
            model.save_weights(weights_path)
            
        models.append(model)
        y_prob = model.predict(x_test) 
        predictions.append(y_prob.argmax(axis=-1))
        acc = model.evaluate(x_test,y_testc)[1]
        accuracies.append(acc)

        print(f"Model: {augmentations[i]} added. Resulting score: {acc}")

    # Results  
        
    # Accuracy vs data augmentation method
        
    print("\n ===== Accuracy vs weight init methods =====")
    accuracy_df = pd.DataFrame(accuracies, columns=["Accuracy"])
    accuracy_df["data_augmentaion_method"] = augmentations
    display(accuracy_df)
    accuracy_df.to_csv(PATH + MODEL_NAME + f"/{run}/accuracy.csv")
        
    print("\n ===== Converting Binary classification =====")
    classified = []
    for prediction in tqdm(predictions):
        classified.append([1 if i==j else 0 for i,j in zip(prediction,y_test)])
        
    ## Correlation between models
    print("\n ===== Correlation =====")  
    correlation_matrix = []

    for ix, x in enumerate(classified):
        row = []
  
        for iy, y in enumerate(classified):
            if (ix == iy):
                row.append(np.nan)
            else:
                row.append(pearsonr(x,y)[0])

        correlation_matrix.append(row)

    correlation_matrix = np.array(correlation_matrix)
    correlation_matrix_df = pd.DataFrame(correlation_matrix)
    correlation_matrix_df.columns = augmentations
    correlation_matrix_df.index = augmentations
    correlation_matrix_df.to_csv(PATH + MODEL_NAME + f"/{run}/correlation_matrix.csv")
    display(correlation_matrix_df)
    correlation = np.nanmean(correlation_matrix.flatten())
    print("Average correlation: " + str(correlation))
    
    
    print("\n ===== Computing ensemble accuracy =====")  
    # Ensemble accuracy
    accuracy_hard = predict(models, x_test, y_testc,voting = 'hard')
    print("Accuracy of ensemble using hard voting: " + str(accuracy_hard))
    accuracy_soft = predict(models, x_test, y_testc,voting = 'soft')
    print("Accuracy of ensemble using soft voting: " + str(accuracy_soft))
    
    
    print("\n ===== Computing ensemble accuracy =====")  
    # Save the results
    file = PATH + MODEL_NAME + f"/results_.csv"
    df = pd.DataFrame([[run,correlation,accuracy_hard,accuracy_soft]])

    if not os.path.isfile(file):
        df.to_csv(file, header=["run", "correlation","accuracy_hard_voting","accuracy_soft_voting"], index=False)
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False)

    clear_output(wait=True)


 ===== Train model: Data augmentation method: rotation  =====
Epoch 1/9999
313/313 [==============================] - 13s 41ms/step - loss: 1.7234 - accuracy: 0.3964 - val_loss: 2.4444 - val_accuracy: 0.1351
Epoch 2/9999
313/313 [==============================] - 12s 38ms/step - loss: 1.3324 - accuracy: 0.5327 - val_loss: 1.6874 - val_accuracy: 0.4096
Epoch 3/9999
313/313 [==============================] - 12s 38ms/step - loss: 1.1738 - accuracy: 0.5900 - val_loss: 1.2279 - val_accuracy: 0.5591
Epoch 4/9999
313/313 [==============================] - 12s 38ms/step - loss: 1.0692 - accuracy: 0.6291 - val_loss: 1.2416 - val_accuracy: 0.5639
Epoch 5/9999
313/313 [==============================] - 12s 37ms/step - loss: 0.9874 - accuracy: 0.6585 - val_loss: 1.0791 - val_accuracy: 0.6161
Epoch 6/9999
313/313 [==============================] - 12s 37ms/step - loss: 0.9215 - accuracy: 0.6816 - val_loss: 1.0317 - val_accuracy: 0.6345
Epoch 7/9999
313/313 [==============================] - 12s 3

,Accuracy,data_augmentaion_method
0,0.6436,rotation
1,0.6435,horizontal shifting
2,0.6800,vetical shifting



 ===== Converting Binary classification =====

100%|██████████| 3/3 [00:00<00:00, 89.96it/s]



 ===== Correlation =====


,rotation,horizontal shifting,vetical shifting
rotation,NaN,0.478407,0.475588
horizontal shifting,0.478407,NaN,0.444532
vetical shifting,0.475588,0.444532,NaN


Average correlation: 0.4661756931452934

 ===== Computing ensemble accuracy =====
Accuracy of ensemble using hard voting: 0.698
Accuracy of ensemble using soft voting: 0.7088

 ===== Computing ensemble accuracy =====


# Results

In [0]:
from scipy import stats

In [0]:
baseline = [0.6636000276,
            0.6371999979,
            0.6478999853,
            0.6585999727,
            0.6401000023,
            0.619599998,
            0.679099977,
            0.6636000276,
            0.6561999917,
            0.6692000031]

In [37]:
augmented = pd.read_csv("/content/CIFAR_data_augmentation/results_.csv")
augmented = augmented.values
augmented

array([[ 1.        ,  0.46764748,  0.6991    ,  0.7089    ],
       [ 2.        ,  0.48367173,  0.7005    ,  0.7198    ],
       [ 3.        ,  0.49942668,  0.7085    ,  0.7181    ],
       [ 4.        ,  0.4463676 ,  0.6859    ,  0.7021    ],
       [ 5.        ,  0.5137241 ,  0.6936    ,  0.7162    ],
       [ 6.        ,  0.47631682,  0.7102    ,  0.7285    ],
       [ 7.        ,  0.47590558,  0.6825    ,  0.7048    ],
       [ 8.        ,  0.49856153,  0.6918    ,  0.7101    ],
       [ 9.        ,  0.48980765,  0.7004    ,  0.7148    ],
       [10.        ,  0.46617569,  0.698     ,  0.7088    ]])

In [0]:
augmented_hard = augmented[:,2]
augmented_soft = augmented[:,3]

In [46]:
np.mean(augmented_hard)

0.69705

In [47]:
np.mean(augmented_soft)

0.71321

In [40]:
stats.ttest_rel(augmented_hard,baseline)

Ttest_relResult(statistic=5.62726513932962, pvalue=0.0003228232881827489)

In [41]:
stats.ttest_rel(augmented_soft,baseline)

Ttest_relResult(statistic=7.616437680629082, pvalue=3.270392678603264e-05)

## Accuracy
The final accuracy of the ensamble on the test set

In [49]:
!zip -r /content/CIFAR_data_augmentation.zip /content/CIFAR_data_augmentation

  adding: content/CIFAR_data_augmentation/ (stored 0%)
  adding: content/CIFAR_data_augmentation/9/ (stored 0%)
  adding: content/CIFAR_data_augmentation/9/correlation_matrix.csv (deflated 50%)
  adding: content/CIFAR_data_augmentation/9/accuracy.csv (deflated 20%)
  adding: content/CIFAR_data_augmentation/9/weights/ (stored 0%)
  adding: content/CIFAR_data_augmentation/9/weights/weights-vetical shifting.h5 (deflated 10%)
  adding: content/CIFAR_data_augmentation/9/weights/weights-horizontal shifting.h5 (deflated 10%)
  adding: content/CIFAR_data_augmentation/9/weights/weights-rotation.h5 (deflated 10%)
  adding: content/CIFAR_data_augmentation/9/history/ (stored 0%)
  adding: content/CIFAR_data_augmentation/9/history/history-vetical shifting.csv (deflated 48%)
  adding: content/CIFAR_data_augmentation/9/history/history-horizontal shifting.csv (deflated 49%)
  adding: content/CIFAR_data_augmentation/9/history/history-rotation.csv (deflated 49%)
  adding: content/CIFAR_data_augmentation